In [2]:
import pandas as pd
import numpy as np
import string
from nltk.stem import WordNetLemmatizer

In [8]:
def format_ingredient(s):
    lm = WordNetLemmatizer()
    printable = string.printable
    PERMITTED_CHARS = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ " 

    filter(lambda x: x in printable, s)
    s = "".join(c for c in s if c in PERMITTED_CHARS)
    s = lm.lemmatize(s.lower())

    return s    

# Given an ingredient, return all subsets of the ingredient string as a list of strings
def get_substrings(ingredient):
    # Base case - Return an empty list of the string is empty
    if ingredient == '':
        return []
    
    word_list = ingredient.split(' ')
    substring_list = []
    substring_list.append(ingredient)
    
    # Base case - Return a list with only the ingredient if there is only one word
    if len(word_list) == 1:
        return substring_list
    
    # Recursive Case
    for i in range(len(word_list)):
        new_list = np.array(word_list, copy=True).tolist()
        new_list.remove(word_list[i])
        substring_word = ' '.join(new_list)
        substring_list.append(substring_word)
        
        sub_list = get_substrings(substring_word)
        for w in sub_list:
            substring_list.append(w)

    return substring_list

# Dedupes the output list of get_substrings function
def get_substrings_deduped(substring_list):
    return np.unique(get_substrings(substring_list)).tolist()

def broadmatch(ing):
    if len(ing.split(' ')) > 10:
        return ing
    
    df = pd.DataFrame(get_substrings_deduped(ing), columns=['ingredient']).set_index('ingredient')
    joined_df = df.join(ing_df, lsuffix = 'l', rsuffix='r')
    normalized_ingredient = joined_df['num_recipes'].idxmax()
    return normalized_ingredient if normalized_ingredient != "nan" else ing

In [4]:
# Read in ingredients
ing_df = pd.read_csv("ingredients.csv").set_index('ingredient')

In [9]:
for i in range(len(ing_df.index)):
    print i, ", ", ing_df.index[i], " --> ", broadmatch(format_ingredient(ing_df.index[i]))

0 ,  salt  -->  salt
1 ,  olive oil  -->  olive oil
2 ,  onions  -->  nan
3 ,  water  -->  water
4 ,  garlic  -->  garlic
5 ,  sugar  -->  sugar
6 ,  garlic cloves  -->  garlic
7 ,  butter  -->  butter
8 ,  ground black pepper  -->  ground black pepper
9 ,  all-purpose flour  -->  flour
10 ,  pepper  -->  pepper
11 ,  vegetable oil  -->  vegetable oil
12 ,  eggs  -->  nan
13 ,  soy sauce  -->  soy sauce
14 ,  kosher salt  -->  salt
15 ,  green onions  -->  onions
16 ,  tomatoes  -->  nan
17 ,  large eggs  -->  eggs
18 ,  carrots  -->  nan
19 ,  unsalted butter  -->  butter
20 ,  ground cumin  -->  ground cumin
21 ,  extra-virgin olive oil  -->  olive oil
22 ,  black pepper  -->  pepper
23 ,  milk  -->  milk
24 ,  chili powder  -->  chili powder
25 ,  oil  -->  oil
26 ,  red bell pepper  -->  pepper
27 ,  purple onion  -->  purple onion
28 ,  scallions  -->  nan
29 ,  grated parmesan cheese  -->  grated parmesan cheese
30 ,  sesame oil  -->  oil
31 ,  corn starch  -->  corn starch
32 , 

230 ,  light soy sauce  -->  soy sauce
231 ,  chopped garlic  -->  garlic
232 ,  sake  -->  sake
233 ,  fresh thyme  -->  thyme
234 ,  smoked paprika  -->  paprika
235 ,  parsley  -->  parsley
236 ,  boneless chicken skinless thigh  -->  chicken
237 ,  grated lemon zest  -->  lemon
238 ,  spaghetti  -->  spaghetti
239 ,  green beans  -->  green beans
240 ,  yellow corn meal  -->  yellow corn meal
241 ,  chile pepper  -->  pepper
242 ,  low sodium chicken broth  -->  chicken broth
243 ,  orange juice  -->  orange
244 ,  fresh ginger root  -->  ginger
245 ,  half & half  -->  nan
246 ,  chicken thighs  -->  chicken
247 ,  sweet onion  -->  sweet onion
248 ,  chopped parsley  -->  parsley
249 ,  shiitake  -->  shiitake
250 ,  corn  -->  corn
251 ,  basmati rice  -->  rice
252 ,  tomatillos  -->  nan
253 ,  chiles  -->  nan
254 ,  pasta  -->  pasta
255 ,  dark brown sugar  -->  sugar
256 ,  chopped fresh mint  -->  fresh mint
257 ,  vinegar  -->  vinegar
258 ,  shredded Monterey Jack chees

465 ,  kidney beans  -->  kidney beans
466 ,  fettucine  -->  fettucine
467 ,  chili flakes  -->  chili
468 ,  wonton wrappers  -->  wonton wrappers
469 ,  asian fish sauce  -->  fish sauce
470 ,  reduced sodium soy sauce  -->  soy sauce
471 ,  lard  -->  lard
472 ,  shredded cheese  -->  cheese
473 ,  kale  -->  kale
474 ,  pecorino romano cheese  -->  cheese
475 ,  roasted peanuts  -->  peanuts
476 ,  light coconut milk  -->  milk
477 ,  shredded carrots  -->  carrots
478 ,  poblano chiles  -->  chiles
479 ,  olives  -->  nan
480 ,  plain flour  -->  flour
481 ,  cardamom pods  -->  cardamom pods
482 ,  sharp cheddar cheese  -->  cheese
483 ,  chicken breast halves  -->  chicken
484 ,  orange zest  -->  orange
485 ,  jasmine rice  -->  rice
486 ,  dry mustard  -->  dry mustard
487 ,  daikon  -->  daikon
488 ,  chipotle chile  -->  chipotle chile
489 ,  grated orange  -->  orange
490 ,  mascarpone  -->  mascarpone
491 ,  fresh tomatoes  -->  tomatoes
492 ,  polenta  -->  polenta
493 ,

737 ,  white beans  -->  beans
738 ,  crabmeat  -->  crabmeat
739 ,  canned black beans  -->  black beans
740 ,  fat skimmed chicken broth  -->  chicken broth
741 ,  low-fat buttermilk  -->  buttermilk
742 ,  ancho powder  -->  ancho powder
743 ,  fillets  -->  nan
744 ,  lime rind  -->  lime
745 ,  sweet italian sausage  -->  italian sausage
746 ,  semi-sweet chocolate morsels  -->  semisweet chocolate
747 ,  hominy  -->  hominy
748 ,  fish fillets  -->  fish
749 ,  rum  -->  rum
750 ,  chicken bouillon  -->  chicken
751 ,  coconut cream  -->  cream
752 ,  salted butter  -->  butter
753 ,  fennel  -->  fennel
754 ,  white sesame seeds  -->  sesame seeds
755 ,  whole cloves  -->  whole cloves
756 ,  chees fresh mozzarella  -->  fresh mozzarella
757 ,  reduced fat milk  -->  milk
758 ,  canned tomatoes  -->  tomatoes
759 ,  artichokes  -->  nan
760 ,  cauliflower florets  -->  cauliflower
761 ,  instant yeast  -->  yeast
762 ,  dried shrimp  -->  shrimp
763 ,  sweetened coconut flakes  

975 ,  lime slices  -->  lime
976 ,  apple juice  -->  juice
977 ,  lobster  -->  lobster
978 ,  chicken bouillon granules  -->  chicken
979 ,  steamed rice  -->  rice
980 ,  California bay leaves  -->  bay leaves
981 ,  sourdough bread  -->  bread
982 ,  gari  -->  gari
983 ,  ground almonds  -->  almonds
984 ,  great northern beans  -->  beans
985 ,  granulated garlic  -->  garlic
986 ,  fruit  -->  fruit
987 ,  seedless cucumber  -->  cucumber
988 ,  capsicum  -->  capsicum
989 ,  plain low-fat yogurt  -->  plain yogurt
990 ,  thai green curry paste  -->  curry paste
991 ,  nonstick spray  -->  nonstick spray
992 ,  crushed ice  -->  ice
993 ,  bread slices  -->  bread
994 ,  turbinado  -->  turbinado
995 ,  wild mushrooms  -->  mushrooms
996 ,  ground round  -->  ground round
997 ,  rocket leaves  -->  leaves
998 ,  organic vegetable broth  -->  vegetable broth
999 ,  colby jack cheese  -->  cheese
1000 ,  salad greens  -->  greens
1001 ,  duck  -->  duck
1002 ,  cod fillets  -->  

...

5819 ,  Lipton® Iced Tea Brew Family Size Tea Bags  -->  tea bags
5820 ,  praline  -->  praline
5821 ,  dark meat  -->  meat
5822 ,  karo  -->  karo
5823 ,  soya flour  -->  flour
5824 ,  hijiki  -->  hijiki
5825 ,  reduced sodium condensed cream of chicken soup  -->  chicken
5826 ,  Pillsbury™ classic pizza crust  -->  pizza crust
5827 ,  red mustard  -->  mustard
5828 ,  pineapple chunks in natural juice  -->  juice
5829 ,  reduced sodium garbanzos  -->  reduced sodium garbanzos
5830 ,  italian vinaigrette  -->  vinaigrette
5831 ,  mineral water  -->  water
5832 ,  pinhead oatmeal  -->  oatmeal
5833 ,  Classico Pasta Sauce  -->  sauce
5834 ,  Earth Balance Buttery Spread  -->  nan
5835 ,  saltpeter  -->  saltpeter
5836 ,  Jose Cuervo  -->  nan
5837 ,  chocolate flavored liqueur  -->  chocolate
5838 ,  mandarin orange juice  -->  orange
5839 ,  meat extract  -->  meat
5840 ,  chocolate cookie crumbs  -->  chocolate
5841 ,  lipton recip secret golden onion soup mix  -->  soup
5842 ,  S

6052 ,  sodium free chicken stock  -->  chicken stock
6053 ,  nut meal  -->  meal
6054 ,  cheese curds  -->  cheese
6055 ,  creamy gravy  -->  gravy
6056 ,  plain chocolate  -->  chocolate
6057 ,  seville oranges  -->  seville oranges
6058 ,  gluten free cooking spray  -->  cooking spray
6059 ,  gomashio  -->  gomashio
6060 ,  tater tots  -->  tater tots
6061 ,  passover wine  -->  wine
6062 ,  sushi vinegar  -->  vinegar
6063 ,  whole wheat uncooked lasagna noodles  -->  noodles
6064 ,  crabmeat frozen  -->  crabmeat
6065 ,  marrons glacés  -->  marrons
6066 ,  mantou  -->  mantou
6067 ,  dumpling dough  -->  dough
6068 ,  liquid sweetener  -->  liquid
6069 ,  wish bone guacamol ranch dress  -->  wish bone ranch dress
6070 ,  brandy extract  -->  brandy
6071 ,  citrus peel  -->  citrus
6072 ,  meat drippings  -->  meat
6073 ,  alaskan king salmon  -->  salmon
6074 ,  paprika paste  -->  paprika
6075 ,  wish-bone light asian sesame ginger vinaigrette dressing  -->  ginger
6076 ,  red c

6263 ,  dried barberries  -->  dried barberries
6264 ,  fully cooked luncheon meat  -->  meat
6265 ,  chocolate liqueur  -->  chocolate
6266 ,  cilantro pesto  -->  cilantro
6267 ,  McCormick Poppy Seed  -->  nan
6268 ,  turkey mince  -->  turkey
6269 ,  whipped dessert topping  -->  whipped topping
6270 ,  roasted bell peppers  -->  roasted bell peppers
6271 ,  reduced sodium refried beans  -->  refried beans
6272 ,  low fat chunky mushroom pasta sauce  -->  sauce
6273 ,  whole wheat english muffins  -->  wheat
6274 ,  Barilla Plus Pasta  -->  pasta
6275 ,  sliced beets  -->  beets
6276 ,  crusty loaf  -->  crusty loaf
6277 ,  vanilla vodka  -->  vanilla
6278 ,  glucose syrup  -->  syrup
6279 ,  gelatin sheet  -->  gelatin
6280 ,  indian flat bread  -->  bread
6281 ,  tuong  -->  tuong
6282 ,  redcurrant jelly  -->  jelly
6283 ,  medium whole wheat tortillas  -->  tortillas
6284 ,  sweetened red beans  -->  beans
6285 ,  poolish  -->  poolish
6286 ,  shiromiso  -->  shiromiso
6287 ,  

6467 ,  hard cheese  -->  cheese
6468 ,  pork picnic roast  -->  pork
6469 ,  jumbo shell pasta , cook and drain  -->  pasta
6470 ,  spiny lobsters  -->  spiny lobsters
6471 ,  warm buns  -->  buns
6472 ,  plum wine  -->  wine
6473 ,  ornamental kale  -->  kale
6474 ,  beef carpaccio  -->  beef
6475 ,  whole grain roll  -->  whole grain roll
6476 ,  demi baguette  -->  baguette
6477 ,  roasted garlic oil  -->  garlic
6478 ,  beluga lentil  -->  beluga lentil
6479 ,  graham flour  -->  flour
6480 ,  mint syrup  -->  mint
6481 ,  veggie patties  -->  veggie patties
6482 ,  honey liqueur  -->  honey
6483 ,  legumes  -->  nan
6484 ,  knorr pasta side   cheesi cheddar  -->  pasta
6485 ,  dry jack  -->  jack
6486 ,  kielbasa (not low fat)  -->  fat
6487 ,  proscuitto  -->  proscuitto
6488 ,  baobab fruit powder  -->  fruit
6489 ,  33% less sodium cooked deli ham  -->  ham
6490 ,  maraschino  -->  maraschino
6491 ,  shank half  -->  shank half
6492 ,  KRAFT Shredded Low-Moisture Part-Skim Moz

6693 ,  citrus vinaigrette  -->  vinaigrette
6694 ,  yam noodles  -->  noodles
6695 ,  lambrusco  -->  lambrusco
6696 ,  kingfish  -->  kingfish
6697 ,  dried raspberry  -->  dried raspberry
6698 ,  poppyseeds  -->  poppyseeds
6699 ,  Crystal Farms Reduced Fat Shredded Marble Jack Cheese  -->  cheese
6700 ,  chile piquin  -->  chile piquin
6701 ,  fiddlehead ferns  -->  fiddlehead ferns
6702 ,  jamaican jerk  -->  jamaican jerk
6703 ,  butternut  -->  butternut
6704 ,  chestnut spread  -->  chestnut spread
6705 ,  one third less sodium chicken broth  -->  chicken broth
6706 ,  low sodium cajun seasoning  -->  cajun seasoning
6707 ,  non-dairy margarine  -->  margarine
6708 ,  wood mushrooms  -->  mushrooms
6709 ,  mojo marinade  -->  marinade
6710 ,  soybean oil  -->  oil
6711 ,  assam  -->  assam
6712 ,  pareve margarine  -->  margarine
6713 ,  bone-in short ribs  -->  nan


In [10]:
broadmatch("i can't believ it' not butter! made with olive oil spread")

"i can't believ it' not butter! made with olive oil spread"